In [28]:
import datetime
import snscrape.modules.twitter as sntwitter
import pandas as pd
from google.cloud import storage
import google.oauth2.service_account
import nltk
import os
import io
import re
from google.cloud import bigquery
import time
import torch.nn.functional as F
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import google.oauth2.service_account
#from google.cloud import bigquery

start_time = time.time()
#creds = google.oauth2.service_account.Credentials.from_service_account_file('/home/code/clef/clef.json',scopes=['https://www.googleapis.com/auth/cloud-platform'])
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/code/clef/clef.json"

start_date = datetime.date.today()  -  datetime.timedelta(days=4)
end_date = datetime.date.today() - datetime.timedelta(days=3)

date="since:{} until:{}".format(start_date, end_date )
client2 = bigquery.Client()
client = storage.Client()
bucket_name = "data_tweet"
bucket = client.bucket(bucket_name)

# Fonction pour écrire les tweets dans un fichier CSV et les stocker dans le bucket Google Cloud Storage
def write_tweets_to_file(tweets,date,crypto):
    # Création d'un dataframe Pandas à partir des tweets
    df = pd.DataFrame(tweets, columns=['Datetime', 'Text','retweet','like','lang'])
    # Filtrage des tweets en anglais
    df = df[df['lang'] == 'en']
    date_of_df=date[6:16]
    df['date'] = pd.to_datetime(df['Datetime']).dt.strftime('%Y-%m-%d')
    df=df[df['date']==date_of_df]
    csv_data = df.to_csv(index=False)
    file_name = "daily_tweet/"+crypto+"/" + str(date_of_df)+".csv"
    bucket.blob(file_name).upload_from_string(csv_data)
    print(f"{file_name} créé avec succès.")
    return df



def get_tweet(crypto,nb_tweet,date):
    tweets_list=[]
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(crypto+' '+str(date) ).get_items()):
        if i>nb_tweet:
            break
        tweets_list.append([tweet.date, tweet.rawContent,tweet.retweetCount,tweet.likeCount,tweet.lang])

    df=write_tweets_to_file(tweets_list,date,crypto)
    return df


nltk.download('stopwords')
ban_words = [  'secret formula','FREE' , 'private access',  'limited spots',  'limited seats',   'make money online',   'be your own boss','easy work',  'earn cash',    'automated system','instant cash',  'quick cash',  'easy cash',    'instant success', 'ðŸ','guaranteed success',  'proven system',  'no experience required',  'no skills needed''no work required',  'no risk',  'zero risk',  'risk-free',  'no investment required','no upfront cost',  'no catch',  'no strings attached',  'limited time offer' ,'Ÿ','last chance',  'exclusive offer',  'special offer',  'secret offer',  'invite only','pre-launch',    'secret strategy',  'hidden strategy', 'secret loophole',  'hidden loophole',  'insider secrets',  'insider info',  'inside knowledge','miracle cure',  'miracle product',  'amazing results',  'instant results',  'mind-blowing results','too good to be true',  'millionaire overnight',  'instant millionaire',  'financial miracle', 'financial breakthrough',  'money miracle',  'money breakthrough',  'secret method',  'hidden method', 'secret technique',  'hidden technique',  'secret formula',  'hidden formula',  'secret blueprint','™','hidden blueprint',  'secret system',  'hidden system',  'secret software',  'hidden software', 'secret tool',  'hidden tool',  'secret strategy',  'hidden strategy',  'secret loophole',  'hidden loophole',  'unlimited income potential','link','TRX']
stopwords_english = set(stopwords.words('english'))


date_generated = [date.strftime("%Y-%m-%d") for date in [start_date + datetime.timedelta(days=x) for x in range((end_date-start_date).days+1)]]

def filter_and_annalyse(df,ban_words=ban_words):

    mask = df['Text'].str.contains('|'.join(ban_words), case=False)
    df = df[~mask]

    url_pattern = re.compile(r'https?://\S+')
    hashtag_pattern = re.compile(r'#\w+')
    mention_pattern = re.compile(r'@\w+')
    digit_pattern = re.compile(r'\b\d+\b')
    stop_words = stopwords.words('english')



    df.loc[:, 'Text'] = df['Text'].apply(lambda x: url_pattern.sub('', x))

    df['Text'] = df['Text'].apply(lambda x: hashtag_pattern.sub('', x))
    df['Text'] = df['Text'].apply(lambda x: mention_pattern.sub('', x))
    df['Text'] = df['Text'].apply(lambda x: digit_pattern.sub('', x))
    df.drop_duplicates(subset=['Text'], keep='first', inplace=True)

    df.drop_duplicates(subset=['Text'],inplace=True)
    return df




def analyze_sentiment(df,date,crypto):

    column = df['Text']
    column = column.apply(lambda x: re.sub(r'[^\w]', ' ', x))
    df['Text'] = column

    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/Finbert")
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/Finbert")

    for i, row in df.iterrows():
        input_ids = torch.tensor(tokenizer.encode(row['Text'], add_special_tokens=True)).unsqueeze(0)
        outputs = model(input_ids)
        logits = outputs[0]
        probs = F.softmax(logits, dim=1)
        sentiment = probs[0][1].item() # La probabilité de la classe positive est utilisée pour la note sur 10
        df.loc[i, 'sentiment'] = sentiment

    csv_data = df.to_csv(index=False)
    date=date[6:16]
    file_name2 = "predictions/"+crypto+"/finbert_ML_pred_" + str(date) +".csv"
    bucket.blob(file_name2).upload_from_string(csv_data)
    return df





def interpreatiton_normalisation(df,crypto):
    from datetime import datetime, timedelta
    df = df.groupby(['date']).mean()
    if crypto == "Bitcoin":
        max_sentiment =0.18867444594931193
        min_sentiment =0.06760287501519527
        max_sentiment_reverse=0.1885925980865344
        min_sentiment_reverse=0.18844601483364987
    elif crypto == "Ethereum":
        max_sentiment = 0.17579640225524298
        min_sentiment = 0.049215803295236206
        max_sentiment_reverse=0.1757341045966489
        min_sentiment_reverse=0.17557387811631814
    elif crypto == "BNB":
        max_sentiment = 0.15699728353315043
        min_sentiment = 0.03457914667716013
        max_sentiment_reverse=0.15695495238604754
        min_sentiment_reverse=0.15680509038373464
    elif crypto == "Dogecoin":
        max_sentiment = 0.15094389359536597
        min_sentiment = 0.03631142479017236
        max_sentiment_reverse=0.15090226891267067
        min_sentiment_reverse=0.15077086288362693

    df['sentiment'] = (100 - df['sentiment']) * (max_sentiment - min_sentiment) / 100 + min_sentiment

    df['sentiment'] = (df['sentiment'] - min_sentiment_reverse) / (max_sentiment_reverse - min_sentiment_reverse) * 100
    df["sentiment"] = df["sentiment"].clip(lower=0, upper=100)

    df.loc[df["sentiment"] < 0, "sentiment"] = 0
    df.loc[df["sentiment"] > 100, "sentiment"] = 100

    df.reset_index(inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    return df
client2 = bigquery.Client()

def ingestion_sentiment(df,crypto):
    print('icic3')
    project = 'sublime-vial-365809'
    df = df.loc[:, ['date', 'sentiment']]

    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("date", bigquery.enums.SqlTypeNames.DATE),
            bigquery.SchemaField("sentiment", bigquery.enums.SqlTypeNames.FLOAT),
        ],
        write_disposition="WRITE_APPEND",
    )
    
    table_id = 'sentiments.'+crypto
    job = client2.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )
    job.result()





def do_the_shit(crypto,nb_tweet,date):
    df = get_tweet(crypto,nb_tweet,date)
    df = filter_and_annalyse(df)
    df = analyze_sentiment(df,date,crypto)
    df = interpreatiton_normalisation(df,crypto)
    #ingestion_sentiment(df,crypto)

print(do_the_shit('BNB',2000,date))
do_the_shit('Bitcoin',5500,date)
do_the_shit('Ethereum',3500,date)
do_the_shit('Dogecoin',2000,date)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victorbarrau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
daily_tweet/BNB/2023-03-11.csv créé avec succès.


/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Text'] = df['Text'].apply(lambda x: url_pattern.sub('', x))
/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].apply(lambda x: hashtag_pattern.sub('', x))
/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:83: SettingWithCopyWarning: 
A value

None
daily_tweet/Bitcoin/2023-03-11.csv créé avec succès.


/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Text'] = df['Text'].apply(lambda x: url_pattern.sub('', x))
/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].apply(lambda x: hashtag_pattern.sub('', x))
/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:83: SettingWithCopyWarning: 
A value

daily_tweet/Ethereum/2023-03-11.csv créé avec succès.


/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Text'] = df['Text'].apply(lambda x: url_pattern.sub('', x))
/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].apply(lambda x: hashtag_pattern.sub('', x))
/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:83: SettingWithCopyWarning: 
A value

daily_tweet/Dogecoin/2023-03-11.csv créé avec succès.


/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Text'] = df['Text'].apply(lambda x: url_pattern.sub('', x))
/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].apply(lambda x: hashtag_pattern.sub('', x))
/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2310911435.py:83: SettingWithCopyWarning: 
A value

In [12]:
import datetime
import snscrape.modules.twitter as sntwitter
import pandas as pd
from google.cloud import storage
import google.oauth2.service_account
import nltk
import os
import io
import re
from google.cloud import bigquery
import time
import torch.nn.functional as F
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import google.oauth2.service_account
#from google.cloud import bigquery


#creds = google.oauth2.service_account.Credentials.from_service_account_file('/home/code/clef/clef.json',scopes=['https://www.googleapis.com/auth/cloud-platform'])
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/code/clef/clef.json"

start_date = datetime.date.today()  -  datetime.timedelta(days=3)
end_date = datetime.date.today()

date="since:{} until:{}".format(start_date, end_date )
#client2 = bigquery.Client()
#client = storage.Client(credentials=creds)
client = storage.Client()
bucket_name = "data_tweet"
bucket = client.bucket(bucket_name)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
date

'since:2023-03-12 until:2023-03-15'

In [30]:
import datetime
import pandas as pd
from google.cloud import storage
import io
from google.cloud import bigquery

client2 = bigquery.Client()
client = storage.Client()
bucket_name = "data_tweet"
bucket = client.bucket(bucket_name)

def get_csv_pred(crypto):
    print('icic')
    date = datetime.date.today()  -  datetime.timedelta(days=5)
    client = storage.Client()
    bucket = client.get_bucket('data_tweet')
    file_name = "predictions/"+crypto+"/finbert_ML_pred_" + str(date) +".csv"
    blob = bucket.get_blob(file_name)
    content = blob.download_as_string()
    df = pd.read_csv(io.StringIO(content.decode('utf-8')))
    df['date'] = pd.to_datetime(date, format='%Y-%m-%d')
    return df
    

def interpreatiton_normalisation(df,crypto):
    from datetime import datetime, timedelta
    print('icic2')

    df = df.groupby(['date']).mean()
    if crypto == "Bitcoin":
        max_sentiment =0.18867444594931193
        min_sentiment =0.06760287501519527
        max_sentiment_reverse=0.1885925980865344
        min_sentiment_reverse=0.18844601483364987
    elif crypto == "Ethereum":
        max_sentiment = 0.17579640225524298
        min_sentiment = 0.049215803295236206
        max_sentiment_reverse=0.1757341045966489
        min_sentiment_reverse=0.17557387811631814
    elif crypto == "BNB":
        max_sentiment = 0.15699728353315043
        min_sentiment = 0.03457914667716013
        max_sentiment_reverse=0.15695495238604754
        min_sentiment_reverse=0.15680509038373464
    elif crypto == "Dogecoin":
        max_sentiment = 0.15094389359536597
        min_sentiment = 0.03631142479017236
        max_sentiment_reverse=0.15090226891267067
        min_sentiment_reverse=0.15077086288362693

    df['sentiment'] = (100 - df['sentiment']) * (max_sentiment - min_sentiment) / 100 + min_sentiment

    df['sentiment'] = (df['sentiment'] - min_sentiment_reverse) / (max_sentiment_reverse - min_sentiment_reverse) * 100
    df["sentiment"] = df["sentiment"].clip(lower=0, upper=100)

    df.loc[df["sentiment"] < 0, "sentiment"] = 0
    df.loc[df["sentiment"] > 100, "sentiment"] = 100

    df.reset_index(inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    return df

def ingestion_sentiment(df,crypto):
    print('icic3')
    project = 'sublime-vial-365809'
    df = df.loc[:, ['date', 'sentiment']]

    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("date", bigquery.enums.SqlTypeNames.DATE),
            bigquery.SchemaField("sentiment", bigquery.enums.SqlTypeNames.FLOAT),
        ],
        write_disposition="WRITE_APPEND",
    )
    
    table_id = 'sentiments.'+crypto
    job = client2.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )
    job.result()




def do_the_shit(crypto):

    df = get_csv_pred(crypto)
    df = interpreatiton_normalisation(df,crypto)
    ingestion_sentiment(df,crypto)


do_the_shit('BNB')
do_the_shit('Bitcoin')
do_the_shit('Ethereum')
do_the_shit('Dogecoin')




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2185407889.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['date']).mean()


icic
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
icic2
icic3


/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2185407889.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['date']).mean()


icic
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
icic2
icic3


/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2185407889.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['date']).mean()


icic
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
icic2
icic3


/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_51875/2185407889.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['date']).mean()
